In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings

In [6]:
# Suppress PyTorch UserWarning about DataLoader iterator re-creation
warnings.filterwarnings("ignore", category=UserWarning)

In [7]:
# --- 0. Configuration ---
CLEANED_CSV_PATH = 'metadata_with_features.csv' 
CHECKPOINT_DIR = './cnn_lstm_hybrid_checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

BATCH_SIZE = 32
EPOCHS = 60 
LEAD_COUNT = 12
SAMPLES = 2500
NUM_CLASSES = 3 
NUM_CLINICAL_FEATURES = 8 
HIDDEN_SIZE = 64
LEARNING_RATE = 2e-4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [8]:
# --- 1. Focal Loss Function (For Imbalance) ---
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        if self.alpha is not None and not isinstance(self.alpha, torch.Tensor):
            self.alpha = torch.tensor(self.alpha, dtype=torch.float32)

    def forward(self, inputs, targets):
        log_p = F.log_softmax(inputs, dim=1)
        log_p = log_p.gather(1, targets.view(-1, 1)).view(-1)
        p = torch.exp(log_p)
        loss = - (1 - p)**self.gamma * log_p
        
        if self.alpha is not None:
            if self.alpha.device != inputs.device:
                self.alpha = self.alpha.to(inputs.device)
            alpha_t = self.alpha.gather(0, targets.view(-1))
            loss = alpha_t * loss
            
        return loss.mean() if self.reduction == 'mean' else loss.sum()

In [9]:
# --- 2. Hybrid CNN-LSTM Model Architecture ---

class CNNLSTM_FeatureExtractor(nn.Module):
    """Processes the raw ECG signal to generate a deep feature vector."""
    def __init__(self, hidden_size=HIDDEN_SIZE, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        
        # CNN (Feature Extraction)
        self.cnn = nn.Sequential(
            nn.Conv1d(LEAD_COUNT, 32, kernel_size=15, stride=2, padding=7),
            nn.BatchNorm1d(32), nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1),
            
            nn.Conv1d(32, 64, kernel_size=11, stride=2, padding=5),
            nn.BatchNorm1d(64), nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1),
            
            nn.Conv1d(64, 128, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm1d(128), nn.ReLU(inplace=True)
        )
        
        # LSTM (Temporal Context)
        self.lstm = nn.LSTM(
            input_size=128, hidden_size=hidden_size, num_layers=num_layers,
            batch_first=True, bidirectional=True
        )

    def forward(self, x):
        cnn_out = self.cnn(x) 
        lstm_input = cnn_out.transpose(1, 2) 
        
        _, (h_n, _) = self.lstm(lstm_input)
        
        final_state = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)
        return final_state

In [10]:
class FinalMultiInputHybridModel(nn.Module):
    """Combines deep features (CNN-LSTM output) and 8 handcrafted features."""
    def __init__(self, num_classes=NUM_CLASSES, num_clinical_features=NUM_CLINICAL_FEATURES):
        super().__init__()
        
        self.signal_extractor = CNNLSTM_FeatureExtractor(hidden_size=HIDDEN_SIZE)
        
        INPUT_SIZE = (2 * HIDDEN_SIZE) + num_clinical_features 
        
        self.final_fc = nn.Sequential(
            nn.Linear(INPUT_SIZE, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes)
        )

    def forward(self, signal, clinical_features):
        deep_features = self.signal_extractor(signal) 
        
        # Concatenate Deep Features with Handcrafted Features (ensuring float type)
        combined_features = torch.cat((deep_features, clinical_features.float()), dim=1) 
        
        return self.final_fc(combined_features)

In [11]:
# --- 3. Custom Dataset for Hybrid Input ---
class HybridECGDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load Signal from absolute path (processed_npy_path)
        try:
            signal = np.load(row['processed_npy_path']).T
            if signal.shape != (LEAD_COUNT, SAMPLES):
                signal = signal[:, :SAMPLES]
            signal = np.nan_to_num(signal, nan=0.0)
        except Exception: 
            signal = np.zeros((LEAD_COUNT, SAMPLES), dtype=np.float32)

        # Load Features from absolute path (feature_path)
        try:
            features = np.load(row['feature_path'])
        except Exception:
            features = np.zeros(NUM_CLINICAL_FEATURES, dtype=np.float32)

        label = {"Low": 0, "Moderate": 1, "High": 2}[row['severity_level']]
        
        return (torch.tensor(signal, dtype=torch.float32), 
                torch.tensor(features, dtype=torch.float32)), label

In [12]:
# --- 4. Evaluation and Checkpointing Functions ---
top_checkpoints = []
N = 5
def save_checkpoint(model, epoch, val_f1):
    global top_checkpoints
    path = os.path.join(CHECKPOINT_DIR, f"hybrid_model_epoch{epoch}.pt")
    torch.save(model.state_dict(), path)
    top_checkpoints.append((val_f1, path))
    top_checkpoints.sort(reverse=True, key=lambda x: x[0])
    if len(top_checkpoints) > N:
        _, worst_path = top_checkpoints.pop()
        if os.path.exists(worst_path):
             os.remove(worst_path)
    print(f"Saved checkpoint: {path} (F1: {val_f1:.4f})")

In [13]:
def hybrid_evaluate(model, dataloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for (signal, features), y in dataloader:
            signal, features = signal.to(DEVICE), features.to(DEVICE)
            logits = model(signal, features)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(y.numpy())
    f1 = f1_score(y_true, y_pred, average='weighted')
    acc = np.mean(np.array(y_true) == np.array(y_pred))
    return acc, f1

In [14]:
# Load the cleaned metadata
df = pd.read_csv(CLEANED_CSV_PATH)

# Split the data
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['severity_level'], random_state=42)

# --- Setup Weights and Sampler ---
train_labels = train_df['severity_level'].map({'Low': 0, 'Moderate': 1, 'High': 2})

In [15]:
# Focal Loss Alpha Weights
class_weights_array = compute_class_weight(class_weight="balanced", classes=np.unique(train_labels), y=train_labels)
alpha_weights = torch.tensor(class_weights_array, dtype=torch.float32).to(DEVICE)

In [16]:
# Sampler Weights (Inverse SQRT frequency for batch balance)
train_labels_np = train_labels.values
class_counts = train_df['severity_level'].value_counts().sort_index()
num_samples = len(train_df)
weights = 1.0 / np.sqrt(class_counts.values) 
sample_weights = weights[train_labels_np]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)

In [17]:
# DataLoaders
train_loader = DataLoader(HybridECGDataset(train_df), batch_size=BATCH_SIZE, sampler=sampler)
val_loader = DataLoader(HybridECGDataset(val_df), batch_size=BATCH_SIZE)

In [18]:
# --- Model, Loss, Optimizer ---
model = FinalMultiInputHybridModel(num_clinical_features=NUM_CLINICAL_FEATURES).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 

criterion = FocalLoss(alpha=alpha_weights, gamma=2.0) 

scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5)

In [20]:
# --- Training Loop ---
print("Starting Final Hybrid Training with Focal Loss...")
best_val_f1 = 0.0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for (x_signal, x_features), y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        x_signal, x_features, y = x_signal.to(DEVICE), x_features.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        
        logits = model(x_signal, x_features)
        loss = criterion(logits, y)

        if torch.isnan(loss): continue
                
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    val_acc, val_f1 = hybrid_evaluate(model, val_loader)
    
    scheduler.step(val_f1)

    print(f"Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")
    save_checkpoint(model, epoch + 1, val_f1)

Starting Final Hybrid Training with Focal Loss...


Epoch 1/60: 100%|██████████| 785/785 [16:01<00:00,  1.22s/it]


Loss: 0.4168 | Val Acc: 0.4912 | Val F1: 0.5747
Saved checkpoint: ./cnn_lstm_hybrid_checkpoints\hybrid_model_epoch1.pt (F1: 0.5747)


Epoch 2/60: 100%|██████████| 785/785 [07:21<00:00,  1.78it/s]


Loss: 0.4052 | Val Acc: 0.3499 | Val F1: 0.4208
Saved checkpoint: ./cnn_lstm_hybrid_checkpoints\hybrid_model_epoch2.pt (F1: 0.4208)


...

Loss: 0.1618 | Val Acc: 0.8086 | Val F1: 0.8182
Saved checkpoint: ./cnn_lstm_hybrid_checkpoints\hybrid_model_epoch58.pt (F1: 0.8182)


Epoch 59/60: 100%|██████████| 785/785 [01:46<00:00,  7.39it/s]


Loss: 0.1459 | Val Acc: 0.8133 | Val F1: 0.8194
Saved checkpoint: ./cnn_lstm_hybrid_checkpoints\hybrid_model_epoch59.pt (F1: 0.8194)


Epoch 60/60: 100%|██████████| 785/785 [01:46<00:00,  7.37it/s]


Loss: 0.1643 | Val Acc: 0.8139 | Val F1: 0.8209
Saved checkpoint: ./cnn_lstm_hybrid_checkpoints\hybrid_model_epoch60.pt (F1: 0.8209)


In [21]:
print("\n--- Final Evaluation ---")
if top_checkpoints:
    best_path = top_checkpoints[0][1]
    model.load_state_dict(torch.load(best_path))
    print(f"Loaded best model from: {best_path}")

model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for (x_signal, x_features), y in val_loader:
        x_signal, x_features = x_signal.to(DEVICE), x_features.to(DEVICE)
        out = model(x_signal, x_features)
        preds = torch.argmax(out, dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(y.numpy())


--- Final Evaluation ---
Loaded best model from: ./cnn_lstm_hybrid_checkpoints\hybrid_model_epoch60.pt


In [ ]:
print("\nFinal Classification Report (Hybrid CNN-LSTM):")
print(classification_report(y_true, y_pred, target_names=["Low", "Moderate", "High"]))


Final Classification Report (Hybrid CNN-LSTM):
              precision    recall  f1-score   support

         Low       0.91      0.88      0.90      5493
    Moderate       0.20      0.13      0.16       190
        High       0.29      0.40      0.33       593

    accuracy                           0.81      6276
   macro avg       0.46      0.47      0.46      6276
weighted avg       0.83      0.81      0.82      6276



In [ ]:
''' 
Classification Report Interpreting:

Support: total number of samples in the validation set
Weighted Average:   F1 score weighted by the size of each class, in this case will always be high due to 'Low' class dominating the score. Poor 
                    metric for imbalanced data.
Macro Average:      Calculates metric for each class independently, and then takes unweighted arithmetic mean of the values.
'''

In [ ]:
'''
The major weakness is the Recall for Moderate (0.13) and High (0.40), meaning the model misses most of the severe cases.

IMPROVE USING:

A. Data Augmentation
You've trained on the same data 60 times. Augmentation creates new, slightly varied training examples without collecting new data, directly 
addressing the data scarcity for minority classes.
Action: 
    Implement Time-Series Augmentations directly in your HybridECGDataset.__getitem__ method (applied only to training data). 
    Useful techniques include:
        Time-Warping: Slightly stretching or compressing the signal.
        Scaling/Jittering: Randomly scaling the amplitude or adding Gaussian noise.
        Random Lead Dropout: Temporarily setting one or two leads to zero to improve robustness.

B. Hyperparameter Tuning (Focal Loss Gamma)
The F1 is high, but the Recall is low, suggesting the model is still finding it "easy" to achieve a good loss value without correctly classifying
the hardest samples.
Action: 
    Increase the Focal Loss gamma parameter (currently 2) to 3 or 4. A higher gamma further penalizes predictions where the model is confident 
    but wrong, forcing it to focus more intensely on the hardest, most ambiguous (often minority) samples.

'''

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import numpy as np
# import pandas as pd
# import os
# from tqdm import tqdm
# from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
# from sklearn.metrics import f1_score, classification_report
# from sklearn.model_selection import train_test_split
# from sklearn.utils.class_weight import compute_class_weight
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# import warnings

# # Suppress PyTorch UserWarning about DataLoader iterator re-creation
# warnings.filterwarnings("ignore", category=UserWarning)

# # --- 0. Configuration ---
# CLEANED_CSV_PATH = 'metadata_with_features.csv' 
# CHECKPOINT_DIR = './cnn_lstm_hybrid_checkpoints'
# os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# BATCH_SIZE = 32
# EPOCHS = 60 
# LEAD_COUNT = 12
# SAMPLES = 2500
# NUM_CLASSES = 3 
# NUM_CLINICAL_FEATURES = 8 
# HIDDEN_SIZE = 64
# LEARNING_RATE = 2e-4
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# DEVICE

# # --- 1. Focal Loss Function (For Imbalance) ---
# class FocalLoss(nn.Module):
#     def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
#         super().__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.reduction = reduction
#         if self.alpha is not None and not isinstance(self.alpha, torch.Tensor):
#             self.alpha = torch.tensor(self.alpha, dtype=torch.float32)

#     def forward(self, inputs, targets):
#         log_p = F.log_softmax(inputs, dim=1)
#         log_p = log_p.gather(1, targets.view(-1, 1)).view(-1)
#         p = torch.exp(log_p)
#         loss = - (1 - p)**self.gamma * log_p
        
#         if self.alpha is not None:
#             if self.alpha.device != inputs.device:
#                 self.alpha = self.alpha.to(inputs.device)
#             alpha_t = self.alpha.gather(0, targets.view(-1))
#             loss = alpha_t * loss
            
#         return loss.mean() if self.reduction == 'mean' else loss.sum()
		
# # --- 2. Hybrid CNN-LSTM Model Architecture ---

# class CNNLSTM_FeatureExtractor(nn.Module):
#     """Processes the raw ECG signal to generate a deep feature vector."""
#     def __init__(self, hidden_size=HIDDEN_SIZE, num_layers=1):
#         super().__init__()
#         self.hidden_size = hidden_size
        
#         # CNN (Feature Extraction)
#         self.cnn = nn.Sequential(
#             nn.Conv1d(LEAD_COUNT, 32, kernel_size=15, stride=2, padding=7),
#             nn.BatchNorm1d(32), nn.ReLU(inplace=True),
#             nn.MaxPool1d(kernel_size=3, stride=2, padding=1),
            
#             nn.Conv1d(32, 64, kernel_size=11, stride=2, padding=5),
#             nn.BatchNorm1d(64), nn.ReLU(inplace=True),
#             nn.MaxPool1d(kernel_size=3, stride=2, padding=1),
            
#             nn.Conv1d(64, 128, kernel_size=7, stride=2, padding=3),
#             nn.BatchNorm1d(128), nn.ReLU(inplace=True)
#         )
        
#         # LSTM (Temporal Context)
#         self.lstm = nn.LSTM(
#             input_size=128, hidden_size=hidden_size, num_layers=num_layers,
#             batch_first=True, bidirectional=True
#         )

#     def forward(self, x):
#         cnn_out = self.cnn(x) 
#         lstm_input = cnn_out.transpose(1, 2) 
        
#         _, (h_n, _) = self.lstm(lstm_input)
        
#         final_state = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)
#         return final_state
		
# class FinalMultiInputHybridModel(nn.Module):
#     """Combines deep features (CNN-LSTM output) and 8 handcrafted features."""
#     def __init__(self, num_classes=NUM_CLASSES, num_clinical_features=NUM_CLINICAL_FEATURES):
#         super().__init__()
        
#         self.signal_extractor = CNNLSTM_FeatureExtractor(hidden_size=HIDDEN_SIZE)
        
#         INPUT_SIZE = (2 * HIDDEN_SIZE) + num_clinical_features 
        
#         self.final_fc = nn.Sequential(
#             nn.Linear(INPUT_SIZE, 64),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.5),
#             nn.Linear(64, num_classes)
#         )

#     def forward(self, signal, clinical_features):
#         deep_features = self.signal_extractor(signal) 
        
#         # Concatenate Deep Features with Handcrafted Features (ensuring float type)
#         combined_features = torch.cat((deep_features, clinical_features.float()), dim=1) 
        
#         return self.final_fc(combined_features)
		
# # --- 3. Custom Dataset for Hybrid Input ---
# class HybridECGDataset(Dataset):
#     def __init__(self, df):
#         self.df = df.reset_index(drop=True)

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
        
#         # Load Signal from absolute path (processed_npy_path)
#         try:
#             signal = np.load(row['processed_npy_path']).T
#             if signal.shape != (LEAD_COUNT, SAMPLES):
#                 signal = signal[:, :SAMPLES]
#             signal = np.nan_to_num(signal, nan=0.0)
#         except Exception: 
#             signal = np.zeros((LEAD_COUNT, SAMPLES), dtype=np.float32)

#         # Load Features from absolute path (feature_path)
#         try:
#             features = np.load(row['feature_path'])
#         except Exception:
#             features = np.zeros(NUM_CLINICAL_FEATURES, dtype=np.float32)

#         label = {"Low": 0, "Moderate": 1, "High": 2}[row['severity_level']]
        
#         return (torch.tensor(signal, dtype=torch.float32), 
#                 torch.tensor(features, dtype=torch.float32)), label
				
# # --- 4. Evaluation and Checkpointing Functions ---
# top_checkpoints = []
# N = 5
# def save_checkpoint(model, epoch, val_f1):
#     global top_checkpoints
#     path = os.path.join(CHECKPOINT_DIR, f"hybrid_model_epoch{epoch}.pt")
#     torch.save(model.state_dict(), path)
#     top_checkpoints.append((val_f1, path))
#     top_checkpoints.sort(reverse=True, key=lambda x: x[0])
#     if len(top_checkpoints) > N:
#         _, worst_path = top_checkpoints.pop()
#         if os.path.exists(worst_path):
#              os.remove(worst_path)
#     print(f"Saved checkpoint: {path} (F1: {val_f1:.4f})")
	
# def hybrid_evaluate(model, dataloader):
#     model.eval()
#     y_true, y_pred = [], []
#     with torch.no_grad():
#         for (signal, features), y in dataloader:
#             signal, features = signal.to(DEVICE), features.to(DEVICE)
#             logits = model(signal, features)
#             preds = torch.argmax(logits, dim=1).cpu().numpy()
#             y_pred.extend(preds)
#             y_true.extend(y.numpy())
#     f1 = f1_score(y_true, y_pred, average='weighted')
#     acc = np.mean(np.array(y_true) == np.array(y_pred))
#     return acc, f1
	
# # Load the cleaned metadata
# df = pd.read_csv(CLEANED_CSV_PATH)

# # Split the data
# train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['severity_level'], random_state=42)

# # --- Setup Weights and Sampler ---
# train_labels = train_df['severity_level'].map({'Low': 0, 'Moderate': 1, 'High': 2})

# # Focal Loss Alpha Weights
# class_weights_array = compute_class_weight(class_weight="balanced", classes=np.unique(train_labels), y=train_labels)
# alpha_weights = torch.tensor(class_weights_array, dtype=torch.float32).to(DEVICE)

# # Sampler Weights (Inverse SQRT frequency for batch balance)
# train_labels_np = train_labels.values
# class_counts = train_df['severity_level'].value_counts().sort_index()
# num_samples = len(train_df)
# weights = 1.0 / np.sqrt(class_counts.values) 
# sample_weights = weights[train_labels_np]

# sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)

# # DataLoaders
# train_loader = DataLoader(HybridECGDataset(train_df), batch_size=BATCH_SIZE, sampler=sampler)
# val_loader = DataLoader(HybridECGDataset(val_df), batch_size=BATCH_SIZE)

# # --- Model, Loss, Optimizer ---
# model = FinalMultiInputHybridModel(num_clinical_features=NUM_CLINICAL_FEATURES).to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 

# criterion = FocalLoss(alpha=alpha_weights, gamma=2.0) 

# scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5)

# # --- Training Loop ---
# print("Starting Final Hybrid Training with Focal Loss...")
# best_val_f1 = 0.0

# for epoch in range(EPOCHS):
#     model.train()
#     total_loss = 0
#     for (x_signal, x_features), y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
#         x_signal, x_features, y = x_signal.to(DEVICE), x_features.to(DEVICE), y.to(DEVICE)
#         optimizer.zero_grad()
        
#         logits = model(x_signal, x_features)
#         loss = criterion(logits, y)

#         if torch.isnan(loss): continue
                
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#         optimizer.step()
#         total_loss += loss.item()

#     avg_loss = total_loss / len(train_loader)
#     val_acc, val_f1 = hybrid_evaluate(model, val_loader)
    
#     scheduler.step(val_f1)

#     print(f"Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")
#     save_checkpoint(model, epoch + 1, val_f1)
	
# 	print("\n--- Final Evaluation ---")
# if top_checkpoints:
#     best_path = top_checkpoints[0][1]
#     model.load_state_dict(torch.load(best_path))
#     print(f"Loaded best model from: {best_path}")

# model.eval()
# y_true, y_pred = [], []
# with torch.no_grad():
#     for (x_signal, x_features), y in val_loader:
#         x_signal, x_features = x_signal.to(DEVICE), x_features.to(DEVICE)
#         out = model(x_signal, x_features)
#         preds = torch.argmax(out, dim=1).cpu().numpy()
#         y_pred.extend(preds)
#         y_true.extend(y.numpy())
		
# print("\nFinal Classification Report (Hybrid CNN-LSTM):")
# print(classification_report(y_true, y_pred, target_names=["Low", "Moderate", "High"]))

